In [1]:
import pandas as pd
import numpy as np
import psycopg2
import json
import requests
import re

In [3]:



def child_asin_count(url,sku):
    
    link='https://www.amazon.'+url+'/dp/'+sku
    print('crawling',link)
    headers = {
        "scheme": "https",
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "accept-encoding": "gzip, deflate, br",
        "accept-language": "en-GB,en-US;q=0.9,en;q=0.8",
        "sec-fetch-dest": "document",
        "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36"}
    #headers = {"user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36"}
    data = requests.get(link, headers=headers)
    try:
        asins_info = re.findall('"dimensionValuesDisplayData"\s*:\s*({.*?})', data.text)

        no_asins = 0
        asin_list = []
        for asins_json in asins_info:
            asins_json = json.loads(asins_json)
            for key, value in asins_json.items():
                if key:
                    no_asins += 1
        print(no_asins)
    except:
        print('\ncrawling failed for:',link)
    
    return no_asins
    

In [4]:
id= input("Enter the SERP Job Keystone id:\n")# 6242907baeaf3c20971706f6
#id=('638f14f0de567323966e8af5','639305d5de567323966f6fce')
client_id= input('client_id:\n')

tablename='client_{0}.serp_data_{1}'.format(client_id,client_id)

query=f"""
select country
from client_resource.client_master
where id ='{client_id}'
"""
dd=pd.read_sql_query(query, conn[1])
country_id=dd['country'][0]
print(country_id)

# getting the crwal dates for SERP Job
query_smd = f"""
select *
from ready.serp_mongo_details
where keystone_job_id= '{id}' """

data = pd.read_sql_query(query_smd, conn[1])
data['crawl_date'] = pd.to_datetime(data['crawl_date'])
print('Following are the crawl dates.\n',data['crawl_date'].sort_values().reset_index().drop('index',axis=1))

#start_date= input('Input starting date in format e.g. 2022-03-29\n')
#end_date=input('Input end date in format e.g. 2022-03-31\n')
print('Still working.\nWAIT...')

#making a list of pipeline ids for given keystone job
#mask = (data['crawl_date'] > start_date) & (data['crawl_date'] <= end_date)
pipe_list=tuple(data['pipeline_id'].tolist())

print("run next Cell")

Enter the SERP Job Keystone id:
6413f7d44cf7bd7022574cab
client_id:
134
MX
Following are the crawl dates.
   crawl_date
0 2023-03-17
1 2023-03-18
2 2023-03-19
3 2023-03-20
4 2023-03-21
5 2023-03-22
6 2023-03-23
Still working.
WAIT...
run next Cell


C:\Users\visha\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\visha\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [5]:
startingdate=input('starting Date')
endingdate= input('Ending Date')

starting Date2023-03-17
Ending Date2023-03-22


In [6]:
Rno=input('Please insert Request Number: ')
Channel=input('Please insert Channel Name: ')

Please insert Request Number: 382
Please insert Channel Name: Super Walmart


In [7]:
data1= data[(data['crawl_date'] >= startingdate) & (data['crawl_date'] <= endingdate)]
pipe_list=tuple(data1['pipeline_id'].tolist())

In [8]:
#fetiching serp data from production table
query = f"""
select *
from ready.serp_details smd join ready.serp_mongo_details m on m.serp_mongo_id=smd.serp_mongo_id
where  smd.crawl_date between '{startingdate}' and '{endingdate}' and keystone_job_id = '{id}'
"""
serp_data = pd.read_sql_query(query, conn[1])
key_list=tuple(serp_data['keyword'].unique())


#getting list of keywords from excel which made manually by coping keys from keystone.
ui=input('Please insert the keywords in the excel sheet with name keystone_keyword.xlsx.\n Loacate the file in same as code.\n If Done Press 1\n if you want to search keywords from feature enter 2\n:')
if ui =='1':
    keystone_key_df=pd.read_excel('keystone_keyword.xlsx')
    keystone_key=keystone_key_df['keyword'].tolist()

elif ui=='2':
    
    query = "select * from entity.keyword_category where country_code='{}' and feature= '{}'".format(country_id, Rno)
    keystone_key_df=pd.read_sql_query(query, conn[1])
    keystone_key=keystone_key_df['keyword'].tolist()
    
else:
    print('Do it and come back.')
print('\nKeywords applied on Keystone SERP Job are=', len(keystone_key))
print('Data in ready--->serp_details  crawled for keywords=', len(key_list))
diff1=len(keystone_key)-len(key_list)
if diff1!=0:
    print("Missing keywords in SERP Details:", (set(keystone_key).difference(key_list)))
else:
    print("Check 1:\nAll keywords applied are present in SERP_Details")
    
    
print("Run Next Cell")

C:\Users\visha\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Please insert the keywords in the excel sheet with name keystone_keyword.xlsx.
 Loacate the file in same as code.
 If Done Press 1
 if you want to search keywords from feature enter 2
:2


C:\Users\visha\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(



Keywords applied on Keystone SERP Job are= 111
Data in ready--->serp_details  crawled for keywords= 93
Missing keywords in SERP Details: {'sabritas mini pack', 'sabritas grandes', 'papas fritas limon', 'chips fuego', 'sabritas bbq', 'nachos', 'papas fritas miguelito', 'chips jalapeño', 'paketaxo', 'sabritas originales', 'chetos mix', 'sabritas flamin hot', 'generico sabritas mini', 'sabritas original', 'flaming hot', 'chetos', 'sabritas naturales', 'rancheritos'}
Run Next Cell


In [9]:
keystone_key_lower=[x.lower() for x in keystone_key]
keystone_key_lower

['takis mix',
 'barcel chipotles',
 'barcel chips',
 'barcel surtido',
 'barcel takis azules',
 'bolsa de mini sabritas',
 'caja de mini sabritas',
 'caja de papas sabritas mini',
 'caja de sabritas surtido mini',
 'cazares frituras',
 'generico sabritas mini',
 'lays surtido de papas',
 'pack de 30 piezas botanas surtidas sabritas',
 'papas sabritas',
 'papas sabritas crema y especias',
 'papas sabritas sabrisurtido con 35pzas',
 'papitas sabritas mini',
 'paquete de sabritas mini surtido',
 'rufless mix',
 'sabritas',
 'sabritas 240',
 'sabritas 54 gr',
 'sabritas 55',
 'sabritas adobadas',
 'sabritas amarillas',
 'sabritas bbq',
 'sabritas blancas',
 'sabritas bolzaza',
 'sabritas caja',
 'sabritas crema y especias',
 'sabritas crujientes moradas',
 'sabritas de limon',
 'sabritas de messi',
 'sabritas especias',
 'sabritas familiar',
 'sabritas flamin',
 'sabritas flamin hot',
 'sabritas grandes',
 'sabritas habaneras',
 'sabritas habanero',
 'sabritas individuales',
 'sabritas jam

In [10]:
keystone_key1= "'" +  ("'" + "','".join(name.replace("'", r"''") for name in keystone_key) + "'") + "'"

In [11]:
query = "select * from entity.keyword_category where country_code='{}' and keyword in ({}) and feature='{}'".format(country_id,str(keystone_key1)[1:-1],Rno)

key_cat=pd.read_sql_query(query, conn[1])


key_cat

C:\Users\visha\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,keyword,category,sub_category,type,brand,country_code,feature,sys_keyword_id,included_keywords,md5_hash,id,sys_kwd_ctg_id,sys_submitted_by,sys_last_updated_on,sys_brand_id,epoch_ms_id,translated_keyword
0,barcel chipotles,food & beverages,corn_,branded,barcel,MX,382,12943669,None,574214053f9eea3ccce6374bd744939d,435706,3,Karan Kumar,2023-03-17 06:31:03.141866,NaN,1.679035e+09,None
1,barcel chips,food & beverages,corn_,branded,barcel,MX,382,12943670,None,d024bb4e3bf54467035b3c80de4a4a77,435707,3,Karan Kumar,2023-03-17 06:31:03.141866,NaN,1.679035e+09,None
2,barcel surtido,food & beverages,corn_,branded,barcel,MX,382,12943671,None,b45e4befad0d3ce0f7faf5cc545119b4,435708,3,Karan Kumar,2023-03-17 06:31:03.141866,NaN,1.679035e+09,None
3,barcel takis azules,food & beverages,corn_,branded,barcel,MX,382,12943672,None,05fded166dfbeab2a502ce3101e94c05,435709,3,Karan Kumar,2023-03-17 06:31:03.141866,NaN,1.679035e+09,None
4,bolitas de queso,food & beverages,corn_,generic,generic,MX,382,12921301,None,1f04008aee067dd284640df36eb507ac,435710,2,Karan Kumar,2023-03-17 06:31:03.141866,450.0,1.679035e+09,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,susalitas enchiladas,food & beverages,corn_,branded,susalitas,MX,382,12943815,None,d200b5d6a9bb5b714eef2899a705f1cd,435812,3,Karan Kumar,2023-03-17 06:31:03.141866,NaN,1.679035e+09,None
107,susalitas nachos,food & beverages,corn_,branded,susalitas,MX,382,12943816,None,0ffa9382cec62b3891c99f0e542112b2,435813,3,Karan Kumar,2023-03-17 06:31:03.141866,NaN,1.679035e+09,None
108,takis mix,food & beverages,corn_,branded,takis,MX,382,12943817,None,3d40cc841cf546d38578eedfdcba3f18,435814,3,Karan Kumar,2023-03-17 06:31:03.141866,NaN,1.679035e+09,None
109,veggie chips,food & beverages,corn_,branded,veggie,MX,382,3809329,None,0bd13bcbeea0bd8042f6d9f42a54c486,435815,3,Karan Kumar,2023-03-17 06:31:03.141866,NaN,1.679035e+09,None


In [12]:
keycat_list=list(key_cat['keyword'].unique())

#keycat_list

l1_l2=(set(keystone_key).difference(keycat_list))

print("Missing keywords in keyword_cat table:", l1_l2)
print('No of keyword missing',len(l1_l2))


remaining_keywords=(list(l1_l2)+list(l1_l2))
remaining_keywords=[x.replace("\'",'"') for x in remaining_keywords]

if len(remaining_keywords)>0:
    query = f""" 
    Select *
    from entity.country_channel_keyword_search_volume
    where keyword in {tuple(remaining_keywords)} and country_code='{country_id}'
    """

    key_vol=pd.read_sql_query(query, conn[1])
    if len(key_vol)>0:
        print(key_vol)
    else:
        print("\nThese keywords are not uploaded in table.\nPlease verify the reason for not uploading of these keywords")

print('Run Next cell....')    

Missing keywords in keyword_cat table: set()
No of keyword missing 0
Run Next cell....


In [13]:
print('Connecting to reporting_data....')        
query = f"""
select *
from {tablename}
where pipeline_id in {pipe_list} 

"""
reporting_data = pd.read_sql_query(query, conn2[1])
reporting_data['brand_name']=reporting_data['brand_name'].str.title().str.strip()


print('connected to Reporting_Data ')
key_reporting=reporting_data['keyword'].unique()

keycat_list_lower=[item.lower() for item in keycat_list]  #to lower case keycat_list

diff2=set(keycat_list_lower).difference(key_reporting)
if len(diff2)==0:
    print('Data for all keywords present in keyword_category table is present in reporting_Data')
else:
    print('\nCheck 2:\nMissing keywords in reporting_data :',set(keycat_list_lower).difference(key_reporting))
    print('No of keywords:',len(set(keycat_list_lower).difference(key_reporting)))

print('\nRun the next cell')

Connecting to reporting_data....


C:\Users\visha\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


connected to Reporting_Data 

Check 2:
Missing keywords in reporting_data : {'sabritas mini pack', 'sabritas grandes', 'papas fritas limon', 'chips fuego', 'nachos', 'sabritas bbq', 'papas fritas miguelito', 'chips jalapeño', 'paketaxo', 'sabritas originales', 'chetos mix', 'sabritas flamin hot', 'generico sabritas mini', 'sabritas original', 'flaming hot', 'chetos', 'sabritas naturales', 'rancheritos'}
No of keywords: 18

Run the next cell


In [14]:

#pipe=['641a9269aa486100af363385','64169de6aa486100af354bbf','64169de6aa486100af354b6c','6417ef67aa486100af3587c4']

#kwds1=['attachments','accessories','professional','ice cream market']

#reporting_data1=reporting_data[~reporting_data['pipeline_id'].isin(pipe)]#!='640808477af12e021d6e00f1']

#reporting_data2=reporting_data1[~reporting_data1['keyword'].isin(kwds1)]#!='63fecdc72b8e105f0c3e65cf']

#reporting_data3=reporting_data2[reporting_data1['pipeline_id']!='powertrain']
# reporting_data3

In [15]:
#reporting_data=reporting_data2

In [16]:
df_dates=data['crawl_date'].sort_values().reset_index().drop('index',axis=1)

df_dates['serp_data_keyword']=df_dates.apply(lambda x: len(reporting_data[reporting_data['crawl_date']==x.crawl_date].keyword.unique()), axis=1 )



serp_data.columns=[*serp_data.columns[:-1],'nitin']

df_dates['serp_details_keyword']=df_dates.apply(lambda x: len(serp_data[serp_data['crawl_date']==x.crawl_date].keyword.unique()), axis=1 )

df_dates['diffrence']=df_dates['serp_details_keyword']-df_dates['serp_data_keyword']
df_dates['MISSING KEYWORD']=df_dates.apply(lambda x: (set(list(serp_data[serp_data['crawl_date']==x.crawl_date].keyword.unique()))).difference(set(list(reporting_data[reporting_data['crawl_date']==x.crawl_date].keyword.unique()))), axis=1 )
df_dates

C:\Users\visha\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)
C:\Users\visha\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)
C:\Users\visha\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable

,crawl_date,serp_data_keyword,serp_details_keyword,diffrence,MISSING KEYWORD
0,2023-03-17,48,48,0,{}
1,2023-03-18,54,54,0,{}
2,2023-03-19,54,54,0,{}
3,2023-03-20,62,62,0,{}
4,2023-03-21,62,62,0,{}
5,2023-03-22,52,52,0,{}
6,2023-03-23,0,0,0,{}


In [17]:
#df_dates.to_excel("check.xlsx")

In [18]:
#reporting_data=pd.read_excel(r"382 serp data amazon.xlsx")

In [19]:
#df_dates.to_excel('vsdvdsv.xlsx')

In [20]:
BRAND=input('\nEnter the Primary Brand----\n')
allbrand=list(key_cat['brand'].unique())
allbrand.sort()

while BRAND not in allbrand:
    print('\nList of Brands:',allbrand)
    BRAND=input('INPUT BRAND WHICH IS PRESENT IN KEYWORD_CAT TABLE IN A LISTED FORMAT ONLY.')
    
    
#forecasting sheet


serp_df=reporting_data
unique_sku_df=reporting_data[['channel_sku_id','sku_title','brand_name']].drop_duplicates('channel_sku_id') #unique sku with title for QC purpose
days=len(reporting_data['crawl_date'].unique()) #crawling days
#
print('Following are the crawl dates present in serp_Data\n',reporting_data['crawl_date'].unique())

#1 Competition width
a=len(serp_df['brand_name'].unique())
a1=serp_df['brand_name'].value_counts()

#2
#top80 % sov
sov_all=serp_df
sov_df2=(sov_all.groupby(['brand_name']).crawl_sv.sum().reset_index(name='sum_sv'))
sov_df2=sov_df2.sort_values('sum_sv', ascending= False)

sov_df2['Label']=np.where(sov_df2['sum_sv'].cumsum()<=sov_df2['sum_sv'].sum() * 0.8,0,1)
df_80sku_all=sov_df2[sov_df2['Label']==0]
allintensity=df_80sku_all['brand_name'].value_counts().reset_index()
allsi=len(allintensity)-1



#3 top 500 asins for product job
sov_organic=serp_df[serp_df['sponsored']!=1]

sov_df1=(sov_organic.groupby(['channel_sku_id']).crawl_sv.sum().reset_index(name='sum_sv'))
sov_df1=sov_df1.sort_values('sum_sv', ascending= False)
sov_df_final=pd.merge(sov_df1,sov_organic[['channel_sku_id','brand_name','sku_title']], on=['channel_sku_id'], how='left')
sov_df_final.drop_duplicates(inplace=True)
sov_df_final.to_excel('Top_ASIN.xlsx', index=False)



#4 top80 % organic
sov_df=(sov_organic.groupby(['brand_name']).crawl_sv.sum().reset_index(name='sum_sv'))
sov_df=sov_df.sort_values('sum_sv', ascending= False)
sov_df['Label']=np.where(sov_df['sum_sv'].cumsum()<=sov_df['sum_sv'].sum() * 0.8,0,1)
df_80sku=sov_df[sov_df['Label']==0]
competition= len(df_80sku['brand_name'].unique())
ocintensity=df_80sku['brand_name'].value_counts().reset_index()
oci=len(ocintensity)



#5
#top80 % sponsered
sov_spon=serp_df[serp_df['sponsored']==1]
sov_df1=(sov_spon.groupby(['brand_name']).crawl_sv.sum().reset_index(name='sum_sv'))
sov_df1=sov_df1.sort_values('sum_sv', ascending= False)

sov_df1['Label']=np.where(sov_df1['sum_sv'].cumsum()<=sov_df1['sum_sv'].sum() * 0.8,0,1)
df_80sku_spon=sov_df1[sov_df1['Label']==0]
osintensity=df_80sku_spon['brand_name'].value_counts().reset_index()
#osintensity=osintensity[osintensity['brand_name']>=1]
osi=len(osintensity)


#10 Brand Keywords hacking
b_keyword=key_cat[key_cat['brand']==BRAND]
df1=pd.merge(sov_spon[['sku_title','brand_name','keyword']],b_keyword[['keyword','brand']], on=['keyword'], how= 'right')
b=len(df1['brand_name'].unique())

#input brand for serp_data
BRANDindata=[item for item in input("Enter the Brand name present in reporting data : ").split('|')]

allbrand1=list(serp_df['brand_name'].unique())
allbrand1= list(filter(None, allbrand1))
#allbrand1.sort()
print(BRANDindata)
while (set(BRANDindata).issubset(set(allbrand1)))==False:
    print('\nList of Brands:',allbrand1)
    BRANDindata=[item for item in input("INPUT BRAND WHICH IS PRESENT IN KEYWORD_CAT TABLE IN A LISTED FORMAT ONLY. with '|'. \n example : puma|nike|shiv naresh : ").split('|')]




#12
sov_organic_top20=serp_df[(serp_df['organic_rank']<=20) & (serp_df['sponsored']!=1)]
sov_organic_top20['brand_name'] = sov_organic_top20['brand_name'].fillna('others')
sov_organic_df_top20=(sov_organic_top20.groupby(['brand_name']).crawl_sv.sum().reset_index(name='sum_sv'))
sov_organic_df_top20=sov_organic_df_top20.sort_values('sum_sv', ascending= False)
try:
    result_12=sov_organic_df_top20[sov_organic_df_top20['brand_name'].isin(BRANDindata)]['sum_sv'].sum() / sov_organic_df_top20['sum_sv'].sum()*100
except:
    result_12=0
    
#13 
sov_sponsored_top20=serp_df[(serp_df['sponsored_rank']<=20) & (serp_df['sponsored']==1)]
sov_sponsored_top20['brand_name'] = sov_sponsored_top20['brand_name'].fillna('others')
sov_df_top20=(sov_sponsored_top20.groupby(['brand_name']).crawl_sv.sum().reset_index(name='sum_sv'))
sov_df_top20=sov_df_top20.sort_values('sum_sv', ascending= False)
try:
    result_13=sov_df_top20[sov_df_top20['brand_name'].isin(BRANDindata)]['sum_sv'].sum() / sov_df_top20['sum_sv'].sum()*100
except:
    result_13=0

#15,16,17
# brand asins
brand_asin=serp_df[serp_df['brand_name'].isin(BRANDindata)]
brand_asin.groupby('channel_sku_id')['rating_count'].max()
brand_asin.drop_duplicates('channel_sku_id', inplace= True)

#16 
avg_brand_rating=brand_asin['rating'].mean()
avg_brand_rating

#
ratings_df=serp_df[['channel_sku_id','sku_title','brand_name','rating','rating_count','crawl_date']]
max_count=ratings_df.groupby(['channel_sku_id','brand_name'])['rating_count'].max().reset_index()
min_count=ratings_df.groupby(['channel_sku_id','brand_name'])['rating_count'].min().reset_index()

#15 Reviews average # per ASIN

pr_brand=max_count[max_count['brand_name'].isin(BRANDindata)].sort_values('rating_count', ascending= False)
pr_brand.dropna(inplace= True)
pr1_brand=pd.merge(pr_brand,unique_sku_df, on=['channel_sku_id'], how='left')

Reviews_average_per_ASIN=pr_brand['rating_count'].sum()



# DIFF OF REVIEWS MAX-MIN
dif_review=pd.merge(min_count,max_count, on=['channel_sku_id'], how= 'outer')
dif_review=dif_review.fillna(0)
dif_review['change_in_rating']=dif_review['rating_count_y']-dif_review['rating_count_x']

#17 New reviews # in last 30 days (on top ASINS)


p_brand=dif_review[(dif_review['change_in_rating']!=0) & (dif_review['brand_name_x'].isin(BRANDindata))].sort_values('change_in_rating', ascending= False)
p1_brand=pd.merge(p_brand,unique_sku_df, on=['channel_sku_id'], how='left')

sumr=p_brand['change_in_rating'].sum()

#19 Price Range
uniquesku=serp_df[['channel_sku_id','sku_title','price','brand_name']].drop_duplicates('channel_sku_id')
nonbrand_asin=uniquesku[~uniquesku['brand_name'].isin(BRANDindata)]
branded_asin=uniquesku[uniquesku['brand_name'].isin(BRANDindata)]

avg_brand_price=branded_asin['price'].mean()
avg_nonbrand_price=nonbrand_asin['price'].mean()
price_range=(avg_brand_price-avg_nonbrand_price)/avg_nonbrand_price*100

# #20
# #best seller 

# client_cat=key_cat[['category','sub_category']].drop_duplicates()
# cct=(list(client_cat['sub_category']))
# cct+=cct
# cct=tuple(cct)
# query=f"""
# select *
# from client_resource.client_category_subcategory
# where client_id ='{client_id}' and sub_category in {cct} 
# """
# c_cat_subcat=pd.read_sql_query(query, conn[1])
# mc_cat_id=list(c_cat_subcat['mc_category_id'])
# mc_cat_id+=mc_cat_id
# mc_cat_id=tuple(mc_cat_id)

# query=f"""
# select *
# from ready.ready_product
# where client_id ='{client_id}' 
# and category_id in {mc_cat_id}
# and crawl_date > now() - interval '3 day '
# """

# ready_product_data=pd.read_sql_query(query, conn[1])
# ready_product_data=ready_product_data.sort_values(['category_id','rank']).reset_index().drop('index',axis=1)

# recent_date=ready_product_data['crawl_date'].max()
# rpd1=ready_product_data[(ready_product_data['crawl_date']==recent_date) & (ready_product_data['rank']<=100)][['category','category_id','asin','title','rank']]

#21

crawl_input=input('Do you want to crawl for child ASINS.\n Press 1, else press any key.\n')
if crawl_input=='1':
    # #21 Product range depth
    url=input('For Child ASINS.\nEnter country-specific TLD’s: for amazon homepagelink.\nexample: \nfor netherland Type---- nl\nfor usa Type---- com \nfor INDIA Type---- in\nfor australia Type--- com.au\n\nwww.amazon.')
    top30_product=sov_df_final[sov_df_final['brand_name'].isin(BRANDindata)].drop_duplicates('channel_sku_id').head(30)
    top30_product.reset_index(drop= True,inplace= True)
    top30_product['child_asin']=top30_product.apply(lambda x: child_asin_count(url,x.channel_sku_id), axis=1)
    
    child=top30_product.replace(0,1)
    product_depth_range=child['child_asin'].sum()/len(child)
    
    child1=pd.merge(child,unique_sku_df, on=['channel_sku_id'], how='left')
else:
    top30_product=pd.DataFrame([["no crawling Done"]],columns = ['Index'])
    product_depth_range=""
    child1=top30_product
    
    #child1=pd.merge(child,unique_sku_df, on=['channel_sku_id'], how='left')

#forecasting sheet
forecasting=[['1','Competition width','How many brands are selling products in the brand’s category?',a-1],
             ['2','Fragmented/Concentrated competition','How many renowned competitors are appearing in the top 80% SOV',allsi],
             ['3','Number of 3rd party sellers','How many 3rd party sellers are selling products in the category?',''],
             ['4','Organic Competition intensity','How many competitors’ brands achieve a high visbility in the top organic results?',oci],
             ['5','Paid Competition intensity','How many competitors’ brands achieve a high visbility in the top paid results?',osi],
             ['6','Paid entry cost Marketplace, GENERIC','Average recommended minimum CPC ?',''],
             ['7','Buy box win rate','What is your brand’s Buy box win rate in %',''],
             ['8','Average AUTHORIZED #sellers selling our brand','What is the number of 3rd party sellers selling the brand’s products',''],
             ['9','Average UNAUTHORIZED #sellers selling our brand','How many non official sellers are selling your brand and products',''],
             ['10','Brand Keywords hacking ','How many brands bid on your brand name',b],
             ['11','Brand’s Product Detailed Page hacking','Are some Brand’s Product detailed Pages hacked by competitors or sellers',''],
             ['12','Organic share of shelves Marketplace','',result_12],
             ['13','Paid share of shelves Marketplace','',result_13],
             ['14','Images & text quality','What is the quality of text and images of the brand’s Product Detailed Page',''],
             ['15','Reviews average # per ASIN','What is the brand’s number of reviews per asin in average',round(Reviews_average_per_ASIN,0)],
             ['16','Reviews average rating score','What is the brand’s review average rating score',round(avg_brand_rating,2)],
             ['17','New reviews # in last {} days (on top ASINS)'.format(days),'How many reviews were posted in the last days in average per asin',round(sumr,0)],
             ['18','A+page','What is the Quality of the brand’s A+ pages if any',''],
             ['19','Price range','How competitive are your Brand’s prices',price_range],
             ['20','Share of best seller rank','How many brand’s SKUs are in the Best sellers',''],
             ['21','Product range depth','What is the depth of the brand’s product range sold on the platform',product_depth_range]]
final_sheet_fc = pd.DataFrame(forecasting, columns = ['Index','Title', 'Definition','Value'])


print('Done\nRun the next cell.')


Enter the Primary Brand----
sabritas
Following are the crawl dates present in serp_Data
 [datetime.date(2023, 3, 22) datetime.date(2023, 3, 20)
 datetime.date(2023, 3, 21) datetime.date(2023, 3, 17)
 datetime.date(2023, 3, 18) datetime.date(2023, 3, 19)]
Enter the Brand name present in reporting data : sabritas
['sabritas']

List of Brands: ['Sabritas', 'Barcel', 'Great  Value', "Nature'S Garden", 'Papabrozas', 'Churrumais', 'Chips', 'Great Value', 'Doritos', 'Bokados', 'Mister', 'Bel Ara', 'Extra Special', 'José Cuervo', 'Sabritones', 'Quesabritas', 'Botanas Lovie´S', 'Snackatelas', 'Lala', 'San Marcos', 'Provi', 'Mc Cain', 'México Lindo', 'Vegetalistos', 'Pringles', 'Nopalia', 'Marketside', 'Cuétara', 'Tyson', 'Paketaxo', 'Cinta De Oro', 'Parma', 'Zwanenberg', 'Tangamanga', 'Bernina', "Wand'S", 'Saladino', 'Paper Craft', 'Kettle', 'Milky Way', 'Mac´Ma', 'Sonric´S', 'Vero', 'Ferrero Collection', 'Veggie Mix', 'Way To Celebrate', 'Tío Nacho', "Pilgrim'S", 'Tio Nacho', 'Bachoco', 'Burr

C:\Users\visha\AppData\Local\Temp\ipykernel_6004\2025656891.py:106: RuntimeWarning: invalid value encountered in double_scalars
  result_13=sov_df_top20[sov_df_top20['brand_name'].isin(BRANDindata)]['sum_sv'].sum() / sov_df_top20['sum_sv'].sum()*100
C:\Users\visha\AppData\Local\Temp\ipykernel_6004\2025656891.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brand_asin.drop_duplicates('channel_sku_id', inplace= True)


Do you want to crawl for child ASINS.
 Press 1, else press any key.
hhhh
Done
Run the next cell.


In [21]:


#to excel file
fileout='Forecasting_'+BRAND+"_"+country_id+"_"+Rno+"_"+Channel+'.xlsx'
with pd.ExcelWriter(fileout, engine='xlsxwriter')as writer:
        final_sheet_fc.to_excel(writer,sheet_name='Forecasting' ,startrow=0,startcol=0,index=False)
        sov_df_final.to_excel(writer,sheet_name='top_asins' ,startrow=0,startcol=0,index=False)
        
        a1.to_excel(writer,sheet_name='1-5' ,startrow=0,startcol=0)
        df_80sku_all.to_excel(writer,sheet_name='1-5' ,startrow=0,startcol=3,index=False)
        df_80sku.to_excel(writer,sheet_name='1-5' ,startrow=0,startcol=7,index=False)
        df_80sku_spon.to_excel(writer,sheet_name='1-5' ,startrow=0,startcol=11,index=False)
        df1.to_excel(writer,sheet_name='10' ,startrow=0,startcol=0,index=False)
        sov_organic_df_top20[['brand_name','sum_sv']].to_excel(writer,sheet_name='12' ,startrow=0,startcol=0,index=False)
        sov_df_top20[['brand_name','sum_sv']].to_excel(writer,sheet_name='13' ,startrow=0,startcol=0,index=False)
        pr1_brand.to_excel(writer,sheet_name='15' ,startrow=0,startcol=0,index=False)
        brand_asin[['channel_sku_id','sku_title','brand_name','price','mrp','rating','rating_count']].to_excel(writer,sheet_name='16' ,startrow=0,startcol=0,index=False)
        p1_brand.to_excel(writer,sheet_name='17' ,startrow=0,startcol=0,index=False)
        uniquesku.to_excel(writer,sheet_name='19' ,startrow=0,startcol=0,index=False)
        nonbrand_asin.to_excel(writer,sheet_name='19' ,startrow=0,startcol=5,index=False)
        branded_asin.to_excel(writer,sheet_name='19' ,startrow=0,startcol=10,index=False)
        #rpd1.to_excel(writer,sheet_name='20' ,startrow=0,startcol=0,index=False)
        child1.to_excel(writer,sheet_name='21' ,startrow=0,startcol=0,index=False)

In [22]:
final_sheet_fc

,Index,Title,Definition,Value
0,1,Competition width,How many brands are selling products in the br...,197
1,2,Fragmented/Concentrated competition,How many renowned competitors are appearing in...,3
2,3,Number of 3rd party sellers,How many 3rd party sellers are selling product...,
3,4,Organic Competition intensity,How many competitors’ brands achieve a high vi...,4
4,5,Paid Competition intensity,How many competitors’ brands achieve a high vi...,0
5,6,"Paid entry cost Marketplace, GENERIC",Average recommended minimum CPC ?,
6,7,Buy box win rate,What is your brand’s Buy box win rate in %,
7,8,Average AUTHORIZED #sellers selling our brand,What is the number of 3rd party sellers sellin...,
8,9,Average UNAUTHORIZED #sellers selling our brand,How many non official sellers are selling your...,
9,10,Brand Keywords hacking,How many brands bid on your brand name,1


In [23]:
print('Data Crawled for {} days'.format(days))
print('1. Competition width----------------------------------------------------------', a-1)
print('2. Fragmented/Concentrated competition ---------------------------------------',allsi)
print('3. Number of 3rd party sellers: ------------- top 500 asins exported for product job.')
print('4. Organic Competition intensity----------------------------------------------',oci)
print('5. Paid Competition intensity-------------------------------------------------',osi)
print('6. Paid entry cost Marketplace, GENERIC')
print('7. Buy box win rate')
print('8. Average AUTHORIZED #sellers selling our brand')
print('9. Average UNAUTHORIZED #sellers selling our brand')
print('10. Brand Keywords hacking-----------------------------------------------------',b)
print('11. Brand’s Product Detailed Page hacking')
print('12. Organic share of shelves Marketplace---------------------------------------',result_12)
print('13. Paid share of shelves Marketplace------------------------------------------',result_13)
print('14. Images & text quality')
print('15. Reviews average # per ASIN-------------------------------------------------',round(Reviews_average_per_ASIN,0))
print('16. Reviews average rating score-----------------------------------------------',round(avg_brand_rating,2))
print('17. New reviews # in last {} days (on top ASINS)-------------------------------'.format(days),round(sumr,0))
print('18. A+page')
print('19. Price range-----------------------------------------------------------------',price_range)
print('20. Share of best seller rank')
print('21. Product range depth--------------------------------------------------------',product_depth_range)


Data Crawled for 6 days
1. Competition width---------------------------------------------------------- 197
2. Fragmented/Concentrated competition --------------------------------------- 3
3. Number of 3rd party sellers: ------------- top 500 asins exported for product job.
4. Organic Competition intensity---------------------------------------------- 4
5. Paid Competition intensity------------------------------------------------- 0
6. Paid entry cost Marketplace, GENERIC
7. Buy box win rate
8. Average AUTHORIZED #sellers selling our brand
9. Average UNAUTHORIZED #sellers selling our brand
10. Brand Keywords hacking----------------------------------------------------- 1
11. Brand’s Product Detailed Page hacking
12. Organic share of shelves Marketplace--------------------------------------- 58.63771389018916
13. Paid share of shelves Marketplace------------------------------------------ nan
14. Images & text quality
15. Reviews average # per ASIN------------------------------------------

In [24]:
conn[0].close()
conn[1].close()
conn2[0].close()
conn2[1].close()

In [ ]:
reporting_data

In [ ]:
reporting_data.to_excel("379 serp data tesco.xlsx")

In [ ]:
pipe_list

In [ ]:
avg_brand_rating=brand_asin['rating'].mean()
avg_brand_rating

In [ ]:
brand_asin['rating']